In [5]:
from ultralytics import YOLO
import cv2
import numpy as np
import os
from tqdm import tqdm

model = YOLO("yolov8n-face.pt")  
print("✔ YOLOv8-face model loaded!")

✔ YOLOv8-face model loaded!


In [6]:
def crop_face_yolo(frame, expand=1.3):
    results = model(frame, verbose=False)[0] 
    if len(results.boxes) == 0:
        return None

    box = results.boxes.xyxy[0].cpu().numpy().astype(int)
    x1, y1, x2, y2 = box

    h, w = frame.shape[:2]

    bw, bh = x2-x1, y2-y1
    pad_w = int((expand - 1) * bw / 2)
    pad_h = int((expand - 1) * bh / 2)

    x1 = max(0, x1 - pad_w)
    y1 = max(0, y1 - pad_h)
    x2 = min(w, x2 + pad_w)
    y2 = min(h, y2 + pad_h)

    return frame[y1:y2, x1:x2]


In [7]:
def process_video_full(video_path, out_dir, sample_frames=20, batch_size=8):
    os.makedirs(out_dir, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    idxs = np.linspace(0, total-1, sample_frames).astype(int)
    idx_set = set(idxs)
    i = 0
    saved = 0
    frames = []
    frame_indices = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if i in idx_set:
            frames.append(frame)
            frame_indices.append(i)
            if len(frames) == batch_size:
                results = model(frames, verbose=False)
                for j, res in enumerate(results):
                    if len(res.boxes) == 0:
                        continue
                    box = res.boxes.xyxy[0].cpu().numpy().astype(int)
                    x1, y1, x2, y2 = box
                    h, w = res.orig_img.shape[:2]
                    bw, bh = x2-x1, y2-y1
                    pad_w = int((1.3 - 1) * bw / 2)
                    pad_h = int((1.3 - 1) * bh / 2)
                    x1 = max(0, x1 - pad_w)
                    y1 = max(0, y1 - pad_h)
                    x2 = min(w, x2 + pad_w)
                    y2 = min(h, y2 + pad_h)
                    crop = res.orig_img[y1:y2, x1:x2]
                    crop = cv2.resize(crop, (299, 299))
                    cv2.imwrite(f"{out_dir}/frame_{saved:03d}.png", crop)
                    saved += 1
                frames = []
                frame_indices = []
        i += 1
    # Process any remaining frames
    if frames:
        results = model(frames, verbose=False)
        for j, res in enumerate(results):
            if len(res.boxes) == 0:
                continue
            box = res.boxes.xyxy[0].cpu().numpy().astype(int)
            x1, y1, x2, y2 = box
            h, w = res.orig_img.shape[:2]
            bw, bh = x2-x1, y2-y1
            pad_w = int((1.3 - 1) * bw / 2)
            pad_h = int((1.3 - 1) * bh / 2)
            x1 = max(0, x1 - pad_w)
            y1 = max(0, y1 - pad_h)
            x2 = min(w, x2 + pad_w)
            y2 = min(h, y2 + pad_h)
            crop = res.orig_img[y1:y2, x1:x2]
            crop = cv2.resize(crop, (299, 299))
            cv2.imwrite(f"{out_dir}/frame_{saved:03d}.png", crop)
            saved += 1
    cap.release()
    return saved


In [ ]:
import concurrent.futures
ROOT = r"FaceForensics++_C23"     # <- change this
OUT  = r"FFprocessed"              # <- change this

folders = [
    "original",
    "Deepfakes",
    "FaceSwap",
    "Face2Face",
    "NeuralTextures",
    "FaceShifter",
    "DeepFakeDetection"
]

N_FRAMES = 20  # recommended
BATCH_SIZE = 64 # You can increase this if you have more GPU memory

def process_one_video(args):
    video_path, save_dir, n_frames, batch_size = args
    os.makedirs(save_dir, exist_ok=True)
    return process_video_full(video_path, save_dir, n_frames, batch_size=batch_size)

for folder in folders:
    in_folder = os.path.join(ROOT, folder)
    out_folder = os.path.join(OUT, folder)
    os.makedirs(out_folder, exist_ok=True)

    videos = [v for v in os.listdir(in_folder) if v.endswith(".mp4")]
    video_args = []
    for vid in videos:
        video_path = os.path.join(in_folder, vid)
        save_dir = os.path.join(out_folder, vid.replace(".mp4", ""))
        video_args.append((video_path, save_dir, N_FRAMES, BATCH_SIZE))

    with concurrent.futures.ThreadPoolExecutor() as executor:
        list(tqdm(executor.map(process_one_video, video_args), total=len(video_args), desc=f"Processing {folder} (parallel)"))

Processing original (parallel):  15%|█▍        | 147/1000 [01:16<05:38,  2.52it/s]